In [2]:
!pip install pyyaml==5.4.1
!pip install darts

     |████████████████████████████████| 636 kB 11.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 370 kB 16.0 MB/s 
     |████████████████████████████████| 793 kB 53.4 MB/s 
     |████████████████████████████████| 1.4 MB 34.6 MB/s 
     |████████████████████████████████| 11.2 MB 52.8 MB/s 
     |████████████████████████████████| 65 kB 3.8 MB/s 
     |████████████████████████████████| 172 kB 55.3 MB/s 
     |████████████████████████████████| 584 kB 61.2 MB/s 
     |████████████████████████████████| 9.8 MB 50.8 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 381 kB 36.1 MB/s 
     |████████████████████████████████| 930 kB 39.7 MB/s 
     |████████████████████████████████| 49 kB 4.8 MB/s 
     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 408 kB

In [1]:
import time
import joblib
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from statsmodels.graphics.tsaplots import plot_acf
from datetime import datetime as dt
from datetime import timedelta, timezone
from scipy.signal import lombscargle, periodogram
import darts
from darts import TimeSeries
from darts.models import KalmanFilter, RNNModel, TCNModel, TransformerModel, NBEATSModel, BlockRNNModel
from darts.utils import timeseries_generation as tg
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Change this as needed locally or in colab
data_relative_path = "/content/drive/MyDrive/Colab Notebooks/glucose_prediction/data"

In [7]:
HDeviceCGM = pd.read_csv(f'{data_relative_path}/HDeviceCGM.txt', sep='|', index_col=False) 

In [8]:
# helper constants
MINUTES_PER_DAY = 1440.0

In [9]:
# helper functions
# TODO: refactor this to a module in ./scripts
def _get_corrected_minutes(uncorrected_total_minutes, minimum_internal_day): 
    return uncorrected_total_minutes - minimum_internal_day*MINUTES_PER_DAY  

def _to_unique_minutes_per_row(row, mininmum_internal_day): 
    time_string = row['DeviceTm']
    day = row['DeviceDtTmDaysFromEnroll']
    x = time.strptime(time_string, "%H:%M:%S")
    total_minutes = timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()/60 + day*MINUTES_PER_DAY
    return _get_corrected_minutes(total_minutes, mininmum_internal_day)

def add_internal_date_time_features(df): 
    minimum_internal_day = df['DeviceDtTmDaysFromEnroll'].min()
    df['TotalDeviceMinutes'] = df.apply(lambda x: _to_unique_minutes_per_row(x, minimum_internal_day), axis=1)
    df['DeviceDateTime'] =  df['TotalDeviceMinutes'].apply(lambda x: pd.to_datetime(x, unit='m'))
    df = df.set_index('DeviceDateTime')
    df = df.sort_values(by='TotalDeviceMinutes', ascending=True, na_position='first')
    df['TotalInternalHours_Diff'] = abs(df['TotalDeviceMinutes'].diff() / 60)
    return df

def save_df_to_hdf(df, patient_id, file_name_ptrn, **kwarg):
    df.to_hdf(file_name_ptrn.format(patient_id), 'df{:02d}'.format(patient_id), **kwarg)
    print(f"...saved {patient_id} to hdf...") 

def get_timestamp_patient_dfs(): 
  timestamp_patient_dfs = {}
  for patient_id in patient_ids:
    # patient_ids less than ten lead with 0, eg: 01
    patient_id_string = f'0{patient_id}' if patient_id < 10 else str(patient_id)
    df = pd.read_hdf(f'{data_relative_path}/timestamp/patient_df_{patient_id_string}')  
    timestamp_patient_dfs
  return timestamp_patient_dfs


Add unique timestamps for each patientID

In [ ]:
CGM =  HDeviceCGM[HDeviceCGM['RecordType'] == "CGM"]
patient_cgm_dfs = {}
for patient_id in CGM['PtID'].unique(): 
  print(patient_id)
  patient_df = CGM[CGM['PtID'] == patient_id]
  patient_df = add_internal_date_time_features(patient_df)
  patient_cgm_dfs[patient_id] = patient_df
  # change this path here as needed
  save_df_to_hdf(patient_df, patient_id, r'/content/drive/MyDrive/Colab Notebooks/data/timestamp/patient_df_{:02}.h5', format='t',
                 complib='blosc', complevel=5)